# Exploration des cahiers de doléance

In [130]:
import re
import csv
import math
import spacy
from statistics import mean, median
from unidecode import unidecode
from collections import defaultdict, Counter

In [2]:
nlp = spacy.load('fr_core_news_lg')

In [3]:
with open('./resources/cahiers.csv') as f:
    data = list(csv.DictReader(f))

In [15]:
for item in data:
    doc = nlp(item['texte_cahier_txt'])
    item['doc'] = doc
    item['sentences'] = [str(sent) for sent in doc.sents]

In [85]:
Counter(item['rural_littoral'] for item in data)

Counter({'rural': 124, 'littoral': 26, '': 13})

In [14]:
QUERY = re.compile(r'\bAngleterre\b')

for item in data:
    for sent in item['sentences']:
        if QUERY.search(sent):
            print('%s (%s) - %s:' % (item['localite'], item['rural_littoral'], item['type_de_cahier']), sent)
            print()

Saint-Jean d’Angély (littoral) - : sur l’établissement des conseils pour les diverses parties, sur les moyens d’augmenter, d’étendre, de favoriser le commerce, d’en rendre la balance favorable à l’État, sur l’examen du dernier traité de commerce avec l’Angleterre, et la comparaison de ses avantages et de ses inconvénients, nos députés demeurent autorisés à adopter les plans qui leur paraîtront les mieux conçus, et les plus propres à assurer la gloire du prince et la prospérité de la nation.

Condac (rural) - paroisse: Représentent aussi lesdits habitants, que pour suppléer à la différence du produit qui se trouvera entre celui des impôts supprimés et celui de l’impôt territorial, il leur paraît convenable d’établir l’impôt du timbre, imposition qui ne portera pas plus de préjudice au commerce de la France qu’elle n’en a porté à celui de l’Angleterre

La Chèvrerie (rural) - paroisse: Représentent aussi lesdits habitants, que pour suppléer à la différence du produit qui se trouvera entre

In [16]:
entities = defaultdict(Counter)
for item in data:
    doc = item['doc']
    for ent in doc.ents:
        if len(ent.text) < 4 or len(ent.text) > 40:
            continue
        entities[ent.label_][ent.text.strip()] += 1

entities

defaultdict(collections.Counter,
            {'LOC': Counter({'Ballans': 4,
                      'Passirol': 1,
                      'Barbezieux': 5,
                      'Chailais': 1,
                      'Saintes': 36,
                      'Collège de Saintes': 1,
                      'Nation': 60,
                      'communauté de Ballans': 1,
                      'Saintonge': 111,
                      'Etats provinciaux': 33,
                      'Guyenne': 10,
                      'La Rochelle': 17,
                      'Etats Provinciaux': 11,
                      'Etats Généraux': 15,
                      'Etat': 118,
                      'Sebilleau': 2,
                      'Généralité': 1,
                      'État': 188,
                      'Versailles': 20,
                      'États Généraux': 2,
                      'Etats': 107,
                      'Royaume': 24,
                      'Clergé': 59,
                      'Sénéchaussée': 4,
     

In [7]:
list(entities.keys())

['LOC', 'PER', 'MISC', 'ORG']

In [8]:
for k, ents in entities.items():
    print(k)
    for item, count in ents.most_common(25):
        print('  •', item, count)
    print()

LOC
  • Angoulême 337
  • État 188
  • Angoumois 180
  • Etat 118
  • Saintonge 111
  • Etats 107
  • États 92
  • Réunion 88
  • États provinciaux 86
  • Nation 60
  • Clergé 59
  • Aunis 57
  • États généraux 52
  • la France 40
  • Aubeterre 40
  • Français 39
  • Saintes 36
  • Blanzac 35
  • Ruffec 35
  • Etats provinciaux 33
  • Angouléme 31
  • Charente 30
  • Poitou 26
  • Royaume 24
  • Rochefort 24

PER
  • Jean 40
  • MM. 32
  • François 30
  • Président 21
  • Généraux 20
  • Sa Majesté 17
  • Messieurs 16
  • Monsieur 15
  • Jacques 12
  • Telles 11
  • S. M. 10
  • Louis XVI 9
  • Saint-Martin 9
  • Brun 9
  • Jean Moreau 9
  • Jean Petit 9
  • Ordres 9
  • Robin 8
  • Thomas 8
  • Jean Mathieu 8
  • la Rochefoucauld 7
  • Pierre Martin 7
  • Jean Arnaud 7
  • Jean Constantin 7
  • Jean Texier 7

MISC
  • Sa Majesté 292
  • États généraux 193
  • Orig 148
  • Texte 105
  • Etats généraux 104
  • Arch 84
  • Comparants 47
  • Fait et arrêté 35
  • Sire 33
  • la Noblesse 2

## Vocabulaires

In [113]:
STOPLIST = {
    'un',
    'une',
    'le',
    'la',
    'l\'',
    'les',
    'de',
    'du',
    'des',
    'd\'',
    'que',
    'qui',
    'qu\'',
    'ce',
    'cette',
    'art',
    'nous',
    'vous',
    'ces',
    'son',
    'sa',
    'ses',
    'leur',
    'et',
    'a',
    'lui',
    'il',
    'ils',
    '-ils',
    'elle',
    'elles',
    'ont',
    'par',
    'pour',
    'sur',
    'dont',
    '-',
    ',',
    ';',
    '.'
}

SQUEEZE_RE = re.compile(r'\s{2,}')

def clean_token(token):
    return SQUEEZE_RE.sub(' ', unidecode(token.lower()).strip())

def noun_fingerprint(chunk):
    chunk = (clean_token(token.text) for token in chunk)
    return SQUEEZE_RE.sub(' ', ' '.join(token for token in chunk if token not in STOPLIST).strip())

In [114]:
DIGITS_RE = re.compile(r'\d')

def filter_noun(token):
    return token and len(token) > 2 and len(token) < 40 and not DIGITS_RE.search(token)

for item in data:
    item['nouns'] = Counter()
    
    for chunk in item['doc'].noun_chunks:
        f = noun_fingerprint(chunk)
        
        if filter_noun(f):
            item['nouns'][f] += 1

In [133]:
dfs = Counter()
N = 0

for item in data:
    if not item['rural_littoral']:
        continue
    
    N += 1
    
    for w, c in item['nouns'].items():
        dfs[w] += 1
        
means = {'rural': defaultdict(list), 'littoral': defaultdict(list)}
        
for item in data:
    if not item['rural_littoral']:
        continue
        
    item['nouns_tfidf'] = Counter()
    
    for noun, count in item['nouns'].items():
        tfidf = count * math.log(N / dfs[noun])
        item['nouns_tfidf'][noun] = tfidf
        means[item['rural_littoral']][noun].append(tfidf)

for t, vocab in means.items():
    for noun, tfidfs in vocab.items():
        vocab[noun] = mean(tfidfs)
    means[t] = Counter(vocab)

In [134]:
for t, vocab in means.items():
    print(t)
    for n, c in vocab.most_common(50):
        print('  •', n, c)
    print()

rural
  • page 25.053176470481276
  • eau-de-vie 20.042541176385022
  • habitans 18.256107358664682
  • cent 15.031905882288767
  • charges publiques 15.031905882288767
  • castin guerin 15.031905882288767
  • pasteurs 15.031905882288767
  • testament 15.031905882288767
  • petite ville 15.031905882288767
  • tresorier 15.031905882288767
  • nevarietur 15.031905882288767
  • deliberants 13.040076684760486
  • mon roi 12.95246434060893
  • demande 11.337324605540518
  • ordre clerge 10.793720283840775
  • paroisse ballans 10.021270588192511
  • chemin 10.021270588192511
  • investissement 10.021270588192511
  • corvee confection grands chemins 10.021270588192511
  • egalite devant impot 10.021270588192511
  • redevance 10.021270588192511
  • registres 10.021270588192511
  • subside 10.021270588192511
  • lesdits commissaires 10.021270588192511
  • trois ordres saintonge 10.021270588192511
  • tribunaux souverains justice 10.021270588192511
  • charges judicature 10.021270588192511
  • m